In [1]:
import numpy as np, pandas as pd, matplotlib as mpl
%matplotlib inline
mpl.rcParams["figure.facecolor"] = "white"
from matplotlib import pyplot as plt


In [2]:
df = pd.read_csv('DS_2_train.csv', index_col=0)
dft = pd.read_csv('DS_2_test.csv', index_col=0)

/fefs/disk/usr100/gentoo/usr/lib64/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df1 = pd.merge(df, pd.read_hdf("rst0.h5", "train"), left_index=True, right_index=True)
dft1 = pd.merge(dft, pd.read_hdf("rst0.h5", "test"), left_index=True, right_index=True)


In [4]:
df1.head()


       event_id             X             Y        Z        TX        TY  \
index                                                                      
0          -999  66162.843750  65620.031250      0.0  0.166890  0.276674   
1          -999  43900.445312  67091.671875      0.0 -0.179967 -0.478795   
2          -999  37564.019531  26618.890625  55599.0 -0.267820 -0.282087   
3          -999  39380.027344  48135.425781  19395.0  0.265197 -0.577493   
4          -999  65040.781250  44018.468750  12930.0 -0.711161 -0.245592   

           chi2  signal  brick_number      rst0  
index                                            
0      2.988714     0.0            44  0.000006  
1      1.901420     0.0            98  0.000025  
2      2.510245     0.0            64  0.019460  
3      1.215219     0.0            63  0.011716  
4      2.973661     0.0             2  0.000203  

In [5]:
dft1.head()

                  X             Y        Z        TX        TY      chi2  \
index                                                                      
0      56472.238281  54979.070312  19395.0 -0.161979 -0.471001  2.681201   
1      70306.289062  42301.472656  37497.0 -0.205394 -0.042299  1.413991   
2      30427.056641  30543.578125  42669.0  0.234635  0.114081  2.702541   
3      45467.257812  27151.515625  71115.0 -0.319327  0.561254  0.593076   
4      59172.652344  33032.457031  41376.0  0.467150 -0.332887  2.236792   

       brick_number      rst0  
index                          
0                68  0.000433  
1                28  0.000448  
2                34  0.000446  
3                28  0.266082  
4                44  0.005904  

In [6]:
# def addf(df):
#     df['eX'] = np.arctan2(1, df.TX)
#     df['rX'] = df.Z*np.cos(df.eX) + df.X*np.sin(df.eX)
#     df['eY'] = np.arctan2(1, df.TY)
#     df['rY'] = df.Z*np.cos(df.eY) + df.Y*np.sin(df.eY)

In [7]:
def addf(z):
    tx2 = z.TX ** 2
    ty2 = z.TY ** 2
    tz2 = 1/(1 + tx2 + ty2)
    bx2 = tx2 * tz2
    by2 = ty2 * tz2
    bxy = np.sqrt(bx2 * by2)
    bz1 = np.sqrt(tz2) + 1

    z['bX'] = (1-bx2/bz1) * z.X - bxy/bz1 * z.Y - z.TX * z.Z
    z['bY'] = - bxy/bz1 * z.X + (1-by2/bz1) * z.Y - z.TX * z.Z

In [8]:
addf(df1)
addf(dft1)

In [9]:
print(np.min(dft1.bX), np.max(dft1.bX))
print(np.min(dft1.bY), np.max(dft1.bY))
print(np.min(df1.bX), np.max(df1.bX))
print(np.min(df1.bY), np.max(df1.bY))


-66070.6158134 134399.878411


-48988.262625 112552.730657


-60619.3305464 137815.430551


-48832.8338511 114820.567293


In [10]:
print(np.min(dft1.X), np.max(dft1.X))
print(np.min(dft1.Y), np.max(dft1.Y))
print(np.min(df1.X), np.max(df1.X))
print(np.min(df1.Y), np.max(df1.Y))


3311.48828125 76710.25


27762.4140625 72240.3359375


3311.48828125 76710.25


27762.4140625 72240.3359375


In [11]:
ft = ['bX', 'bY']
bbX = np.linspace(-48989, 114821, 81)
bbY = np.linspace(-66071, 137816, 81)
edges = (bbX, bbY)
ed = [edges[i][1] - edges[i][0] for i in (0,1)]

def cnt4d(df, kern=[1]):
    H, e_ = np.histogramdd(df[ft].values, bins=edges,
                           weights=df.rst0.values)
    il = [np.floor((df[ft[i]].values - edges[i][0])/ed[i]).astype(np.int) for i in (0,1)]
    
    rst = {'hb{}'.format(i): gaussian_filter(H, i)[il[0], il[1]] for i in kern}
    return pd.DataFrame(rst, index=df.index)

In [31]:
ftn = ['X', 'Y']
iX = np.linspace(27762, 72241, 81)
iY = np.linspace(3311, 76711, 81)
edgesn = (iX, iY)
edn = [edgesn[i][1] - edgesn[i][0] for i in (0,1)]

def cnn(df, kern=[1]):
    H, e_ = np.histogramdd(df[ftn].values, bins=edgesn,
                           weights=df.rst0.values)
    il = [np.floor((df[ftn[i]].values - edgesn[i][0])/edn[i]).astype(np.int) for i in (0,1)]

    rst = {'rb{}'.format(i): gaussian_filter(H, i)[il[0], il[1]] for i in kern}
    return pd.DataFrame(rst, index=df.index)


In [13]:
def f_plot(df, col="chi2"):
    plt.figure(dpi = 80, figsize=(5,4))
    first = plt.hist(df.loc[df.signal == True, col], normed = True, histtype="step", bins=10, label = 'signal')
    second = plt.hist(df.loc[df.signal == False, col], normed = True, histtype="step", bins=10, label = 'background')
    plt.title('{} distribution'.format(col))
    plt.legend()


In [14]:
dfx = df1.query("brick_number==57")
H, e_ = np.histogramdd(dfx[ftn].values, bins=edgesn, weights=dfx.rst0.values)
dfx1 = dfx.query("signal > 0.5 & rst0 < 0.5")

# dfx = df.query("brick_number==1")
# x = cnt4d(dfx)
# plt.plot(dfx.signal, x, '.', alpha=0.01)
# plt.show()


In [15]:
from scipy.ndimage import gaussian_filter
plt.imshow(H.T, origin="lower", extent=(np.min(e_[0]), np.max(e_[0]), np.min(e_[1]), np.max(e_[1])))
plt.plot(dfx1.X, dfx1.Y, 'x', color="red")
plt.show()
plt.imshow(gaussian_filter(H.T, 5), origin="lower", extent=(np.min(e_[0]), np.max(e_[0]), np.min(e_[1]), np.max(e_[1])))
plt.plot(dfx1.X, dfx1.Y, 'x', color="red")


In [16]:
gaussian_filter(H, 2).shape

(80, 80)

In [17]:
dfx = df1.query("brick_number==57")
dfx['x'] = cnt4d(dfx)
dfx1 = dfx.query("signal > 0.5 & rst0 < 0.5")
f_plot(dfx, "x")
plt.hist(dfx1.x, histtype="step", normed=True, label="outlier")
plt.show()


/fefs/disk/usr100/gentoo/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
dfx = df1.query("brick_number==99")
dfx['x'] = cnt4d(dfx)
f_plot(dfx, "x")
dfx['y'] = cnn(dfx)
f_plot(dfx, "y")
plt.show()


/fefs/disk/usr100/gentoo/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/fefs/disk/usr100/gentoo/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [27]:
scales = [0,1,2,4,8]

In [39]:
xdf = df1.groupby("brick_number").apply(cnt4d, scales)


In [40]:
dn = df1.groupby("brick_number").apply(cnn, scales)

In [41]:
pd.concat([xdf,dn], axis=1).to_hdf("kernel.h5", "train")

In [42]:
xdft = dft1.groupby("brick_number").apply(cnt4d, scales)


In [43]:
dnt = dft1.groupby("brick_number").apply(cnn, scales)


In [44]:
pd.concat([xdft,dnt], axis=1).to_hdf("kernel.h5", "test")


Debug plots:

In [141]:
d2 = dft.query('brick_number==1')
d2s = d2.query('signal > 0.5')
NN=100

In [142]:
c, bx, by, p = plt.hist2d(d2s['rX'], d2s['rY'], bins=NN)
plt.show()



In [143]:
c, bx, by, p = plt.hist2d(d2s['X'], d2s['Y'], bins=NN, weights = d2s.signal)
plt.show()


In [3]:
df['brick_number'].unique()

array([ 44,  98,  64,  63,   2,  68,  60,  70,  30,  51,  19,  66,  26,
        94,  43,  20,   6,  31,  75,  67,  33,   4,  77,  15,  76,  36,
        71,  10,  91,  85,  27,  37,  89,  18,  92,  61,  29,  48,  28,
        54,  57,  72,  88,  17,  80,  14,   9,  95,  32,  22,  58, 100,
        81,  52,  34,  84,  74,  65,  12,  41,  87,  40,  56,  21,  78,
        46,   7,   5,  90,  45,  79,  24,  99,  16,   3,  62,  97,   1,
        69,  42,  11,  53,  96,  55,  73,  49,  86,  35,  59,  83,  13,
        82,  93,  39,  25,  38,  50,  23,   8,  47])

In [6]:
df.query("brick_number<1 | brick_number >=21").head()

       event_id             X             Y        Z        TX        TY  \
index                                                                      
0          -999  66162.843750  65620.031250      0.0  0.166890  0.276674   
1          -999  43900.445312  67091.671875      0.0 -0.179967 -0.478795   
2          -999  37564.019531  26618.890625  55599.0 -0.267820 -0.282087   
3          -999  39380.027344  48135.425781  19395.0  0.265197 -0.577493   
5          -999  55179.875000  24942.843750  41376.0 -0.173363 -0.318434   

           chi2  signal  brick_number  
index                                  
0      2.988714     0.0            44  
1      1.901420     0.0            98  
2      2.510245     0.0            64  
3      1.215219     0.0            63  
5      2.488240     0.0            68  

In [10]:
N = 5
list(range(1, 101, 100//N))

[1, 21, 41, 61, 81]

In [11]:
dfb = df.set_index("brick_number")

In [17]:
dfb.loc[1]

KeyError: 'Cannot get left slice bound for non-unique label: 1'

In [19]:
df.head()

       event_id             X             Y        Z        TX        TY  \
index                                                                      
0          -999  66162.843750  65620.031250      0.0  0.166890  0.276674   
1          -999  43900.445312  67091.671875      0.0 -0.179967 -0.478795   
2          -999  37564.019531  26618.890625  55599.0 -0.267820 -0.282087   
3          -999  39380.027344  48135.425781  19395.0  0.265197 -0.577493   
4          -999  65040.781250  44018.468750  12930.0 -0.711161 -0.245592   

           chi2  signal  brick_number  
index                                  
0      2.988714     0.0            44  
1      1.901420     0.0            98  
2      2.510245     0.0            64  
3      1.215219     0.0            63  
4      2.973661     0.0             2  

In [27]:
df.sample(1)


         event_id             X            Y        Z        TX        TY  \
index                                                                       
5981325      -999  64375.707031  4412.222656  58185.0 -0.182068 -0.236747   

             chi2  signal  brick_number  
index                                    
5981325  2.237713     0.0            87  